In [ ]:
# Imports 
from IPython.display import Image, display
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import interact

import sys

# Add src folder
# TODO: this can be removed after a pip install
cwd = Path.cwd()
sys.path.insert(0, str(cwd.parents[0] / 'src'))

# Import wrapper code for CellPose and check which device to use
import utils_segmentation

In [ ]:
# >>>>> Various widgets to define input parameters

# Textbox for path to be scanned
textbox_path_process = widgets.Text(
    #placeholder='path-to-folder-with-data',
    value=r'D:\Documents\Data\test-data\cell-pose\jupyter-tests\example_data\acquisition',
    description='Data path:'
)

# Textbox for path to be scanned
textbox_path_save = widgets.Text(
    #placeholder='path-to-folder-to-save-data',
    value=r'D:\Documents\Data\test-data\cell-pose\jupyter-tests\example_data\analysis\segmentation-input',
    description='Results path:'
)

# Textbox to specify string for cytoplasmic images
textbox_channel_ident = widgets.Text(
    value='dapi',
    description='Channel str:'
)

dropdown_projection_type = widgets.Dropdown(
    options=['mean', 'max', 'indiv'],
    value='mean',
    description='Projection:',
    disabled=False,
)

In [ ]:
# >>>> Assemble widget for segmentation of cells and nuclei

# Button to perform prediction
button_preprocess = widgets.Button(
    description='Pre-process data',
    tooltip='Click to pre-process data',
    icon='angle-double-right'
)

# Define predict function
def process(b=None):
    path_process = textbox_path_process.value
    path_save = textbox_path_save.value
    channel_ident = textbox_channel_ident.value
    projection_type = dropdown_projection_type.value

    # Path containing the data
    if len(path_process) == 0:
        print('Path containing data to be processed has to be specified.')
        return
    
    path_process = Path(path_process)
    
    if not path_process.is_dir():
        print('Path containing data to be processed does not exist.')    
        return
    
    # Path to save data
    if len(path_save) == 0:
        print('Path containing data to be segmented has to be specified.')
        return
    
    path_save = Path(path_save)
    if not path_save.is_dir():
        path_save.mkdir(parents=True)
            
    # Perform segmentation
    print('Pre-processing is performed ... for more details see tab \'Log\'.')
        
    # Call segmentation routine
    with tab_out:
        utils_segmentation.folder_prepare_prediction(
                          path_process=path_process,
                          search_type=None,
                          channel_ident=channel_ident,
                          projection_type=projection_type,
                          path_save=path_save)
    print('Processing finished.')
    
    #  Show results
    print('\nPre-processing is performed:')
    @interact
    def show_images(file=path_save.glob(f'*{channel_ident}*.png')):
        if file: 
            display(Image(file))
        
    
# Assign to button
@button_preprocess.on_click
def plot_on_click(b):
    process()
    
    
# Arrange controls
tab_process = widgets.VBox(children=[
                          textbox_path_process,
                          textbox_path_save,    
                          textbox_channel_ident,
                          dropdown_projection_type,
                          button_preprocess

                      ])

# Tab for log
tab_out = widgets.Output(layout={'border': '1px solid black'})

# Entire interface
ui_preprocess = widgets.Tab(children=[tab_process, tab_out])
ui_preprocess.set_title(0, 'Process')
ui_preprocess.set_title(1, 'Log')